In [ ]:
import sqlite3
con = sqlite3.connect('SQL.db')
cur = con.cursor()
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
import numpy as np
import nltk
import pandas as pd

### Inicialmente criamos a tabela words, que irá conter todas as palavras do nosso vocabulário. Em sequência criamos a tabela website_has_word, que é uma tabela que relaciona uma determinada palavra a um site, e seu respectivo Tfidf

In [ ]:
# cur.execute("CREATE TABLE words (word_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, word TEXT)")
# cur.execute("CREATE TABLE website_has_word (whw_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, word_id INT, website_id INT, tfidf FLOAT)")


In [ ]:
con.commit()

### A partir de todos os textos disponíveis, criamos a matriz esparsa de Tfidfs e a transformamos em um dataframe

In [ ]:
all_data = []
for r in cur.execute("SELECT website_id, website_content FROM websites"):
    website_id = r[0]
    website_content = r[1]
    all_data.append(r[1])


In [ ]:
vec = TfidfVectorizer(min_df = 5)

In [ ]:
data = vec.fit_transform(all_data)

In [ ]:
tfidf_df = pd.DataFrame(data.toarray(), columns=vec.get_feature_names())


In [ ]:
tfidf_df.head()

### Com esse dataframe criamos um set com todas palavras únicas, removendo símbolos e caracteres indesejados, e as inserimos na tabela word

In [ ]:
all_words = set()
for i in tfidf_df.columns:
    clean_word = re.sub("[^A-Za-z#(\w+)]", "", i);
    all_words.add(clean_word.lower())

In [ ]:
len(all_words)

In [ ]:
for word in all_words:
    cur.execute(f"INSERT INTO words (word) VALUES ('{word}')")
    con.commit()

### Finalmente, utilizamos o dataframe de Tfidfs e a tabela words para preencher a tabela website_has_word

In [ ]:
word_ids = {}
for word in tfidf_df.columns:
    r =cur.execute(f"SELECT word_id FROM words WHERE word = '{word}'")
    s = cur.fetchall()
    if len(s) > 0:
        word_ids[word] = s[0][0]
        
        


In [ ]:
insert_var = []
i = 1
for index, row in tfidf_df.iterrows():
    for e in range(len(row)):
        if row[e] != 0:
            clean_word = re.sub("[^A-Za-z#(\w+)]", "", row.index[e]);
            insert_var.append((word_ids[clean_word], i, row[e]))
    insert_var_map = ','.join(list(map(str ,insert_var)))
    if len(insert_var_map) != 0:
        cur.execute(f"INSERT INTO website_has_word (word_id, website_id, tfidf) VALUES {insert_var_map}")
        con.commit()
        insert_var = []
        print(f"Document{i} finished")
    i+=1

